### 2014-1 2do Recuperatorio
Se tiene un archivo distribuido con información de una red social en la cual cada registro tiene información sobre un usuario y una lista de sus amigos y sus enemigos (user_id,vector_ids_amigos,vector_ids_enemigos). Queremos encontrar al usuario que figura en mas listas de enemigos, para designarlo el enemigo público número uno.
(8 pts por programar lo pedido) (5 pts por programar la agregación necesaria) (2 pts indique cuantos procesos reducers usaría)

In [3]:
# (user_id,vector_ids_amigos,vector_ids_enemigos)
social_net = [
    (1, [2, 3], [4, 5, 6]),
    (2, [3, 4], [5, 6]),
    (3, [2, 1], [7, 6]),
    (4, [6, 7, 1], [6]),
    (5, [1, 2, 3, 4], [7]),
    (6, [7], [1, 2, 3, 4]),
    (7, [6], [1, 5, 4, 6])
]

In [14]:
# Usuario que figura en mas listas de enemigos
socialRDD = sc.parallelize(social_net)
socialRDD.flatMap(lambda x: x[2])\
         .map(lambda x: (x, 1))\
         .reduceByKey(lambda a, b: a + b)\
         .reduce(lambda a, b: a if a[1] > b[1] else b)[0]

6

---
### 2016-1 2do Recuperatorio
Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id,user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón. 

In [1]:
# (chat_id,user_id, nickname, text)
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias?'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [5]:
# Usuario mas pregunton
chatRDD = sc.parallelize(chats)
chatRDD.map(lambda x: (x[1], x[3].count('?')))\
       .reduceByKey(lambda a, b: a + b)\
       .reduce(lambda a, b: a if a[1] > b[1] else b)[0]

1

---
### 2014-1 1er Parcial
Se tiene un archivo con información sobre visitas a páginas web de la forma: (URL, visitas, fecha). Existe solo un registro por día para cada URL. Se quiere generar un archivo que, por cada URL, indique cuál fue la fecha en la que tuvo mas visitas y la cantidad de visitas.
Programar lo pedido en Map Reduce usando agregación para minimizar la cantidad de datos que deben transferirse en la red.
Atención: La resolución es muy simple, trivial, asi que es fundamental resolver la agregación para el puntaje completo.

In [6]:
# (URL, visitas, fecha)
visits = [
    ('reddit.com',99999, 20160812),
    ('reddit.com', 9999999, 20160810),
    ('reddit.com', 999999999, 20160809),
    ('reddit.com', 99999999999, 20160808),
    ('reddit.com/r/argentina', 45000000, 20160812),
    ('reddit.com/r/argentina', 450000, 20160810),
    ('reddit.com/r/argentina', 4500, 20160809),
    ('reddit.com/r/gamedev', 2, 20160812),
    ('reddit.com/r/gamedev', 1, 20160810),
    ('reddit.com/r/gamedev', 0, 20160809),
    ('reddit.com/r/programmerhumor', 1, 20160812)
]

In [16]:
# Cuál fue la fecha en la que tuvo mas visitas y la cantidad de visitas
visitsRDD = sc.parallelize(visits)
visitsRDD.map(lambda x: (x[0], (x[1], x[2])))\
         .reduceByKey(lambda a, b: a if a[0] > b[0] else b).collect()

[('reddit.com/r/gamedev', (2, 20160812)),
 ('reddit.com/r/programmerhumor', (1, 20160812)),
 ('reddit.com/r/argentina', (45000000, 20160812)),
 ('reddit.com', (99999999999, 20160808))]

---
### 2015-2 2do Recuperatorio
Un telescopio registra automaticamente la luminosidad de distintas estrellas generando un RDD con registros de tipo (star_id,magnitude,spectral_type, timestamp). Queremos obtener un listado de estrellas que tienen el mismo tipo espectral e igual promedio de magnitud una vez redondeado el mismo a un decimal. El resultado debe ser una lista en donde cada elemento de la lista es una lista de ids de estrellas similares.

In [1]:
# (star_id,magnitude,spectral_type, timestamp)
stars = [
    (1, 5, 1, 1),
    (2, 10, 1, 1),
    (3, 6, 1, 1),
    (4, 5.5, 2, 1),
    (1, 6, 1, 2),
    (2, 9, 1, 2),
    (3, 5, 1, 2),
    (1, 5.5, 1, 3),
    (2, 11, 1, 3),
    (3, 5.5, 1, 3)
]

In [12]:
# Listas de ids de estrellas con mismo tipo espectral y promedio de magnitud
starsRDD = sc.parallelize(stars)
starsRDD.map(lambda x: (x[0], (x[2], x[1], 1)))\
        .reduceByKey(lambda a, b: (a[0], a[1] + b[1], a[2] + b[2]))\
        .map(lambda x: ((x[1][0], round(x[1][1]/x[1][2], 1)) ,[x[0]]))\
        .reduceByKey(lambda a, b: a + b)\
        .collect()

[((2, 5.5), [4]), ((1, 10.0), [2]), ((1, 5.5), [1, 3])]

---
### 2015-2 1er Parcial
Se tiene un RDD con las coordenadas de rectángulos de la forma (x1,x2,y1,y2). Se pide programar en PySpark un programa que encuentre el rectángulo de superficie mínima que contiene al punto (w,z)

```
y2   -------------
    |             |
    |             |
y1   -------------

     x1         x2
```

In [20]:
# (w, z)
w = 1
z = 1

# (x1, x2, y1, y2)
rectangles = [
    (0, 4, 0, 40),
    (0, 4, 0, 4),
    (20, 30, 50, 60),
    (150, 153, 12, 74)
]

In [24]:
rectanglesRDD = sc.parallelize(rectangles)
rectanglesRDD.filter(lambda x: (x[0] < w < x[1]) and (x[2] < z < x[3]))\
             .map(lambda x: (x, abs(x[1] - x[0]) * abs(x[3] - x[2])))\
             .reduce(lambda a, b: a if a[1] < b[1] else b)[0]

(0, 4, 0, 4)

---
### Simil 2016-1 1er Parcial
Dado un rdd con registros (Padron, Codigo de Materia, Codigo de Curso, Profesor a Cargo, Nota)
Hacer un programa en PySpark que liste para cada curso de cada materia, el promedio de notas de los alumnos que aprobaron la misma. El listado debe contener Codigo de Materia, Codigo de Curso, Profesor a cargo y Promedio de Notas, y debe estar ordenado por materia. Tener en cuenta solo los cursos que tengan al menos 2 alumnos aprobados.

In [17]:
# (Padron, Codigo de Materia, Codigo de Curso, Profesor a Cargo, Nota)
# Sup que el profesor a cargo es fijo segun el codigo de curso
alumni = [
    (1, 5, 1, 'Rosita', 1),
    (2, 6, 2, 'Luis', 2),
    (3, 6, 1, 'Diego', 3),
    (4, 5, 2, 'Pablo', 4),
    (5, 6, 1, 'Diego', 5),
    (6, 5, 2, 'Pablo', 6),
    (7, 5, 1, 'Rosita', 7),
    (3, 5, 2, 'Pablo', 8),
    (2, 6, 1, 'Diego', 9),
    (1, 5, 2, 'Pablo', 10)
]

In [18]:
alumniRDD = sc.parallelize(alumni)
# (Mapeamos registros a (cod_materia, cod_curso), (profesor, nota, 1))
# Y terminamos con registros (cod_materia, cod_curso, profesor, promedio_notas_aprobados)
alumniRDD.map(lambda x: ((x[1], x[2]), (x[3], x[4], 1)))\
         .filter(lambda x: x[1][1] >= 4)\
         .reduceByKey(lambda a, b: (a[0], a[1] + b[1], a[2] + b[2]))\
         .filter(lambda x: x[1][2] >= 2)\
         .map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]/x[1][2]))\
         .sortBy(lambda x: x[0])\
         .collect()

[(5, 2, 'Pablo', 7), (6, 1, 'Diego', 7)]

---
### Simil Parcial 2014-2 2do Recuperatorio
 Se tiene un rdd con datos de la forma (url_id,user_id) representando visitas a un website. (Se pueden repetir registros). Se pide programar en PySpark un script que indique cual es el promedio global de sitios visitados por usuario.

In [4]:
# (url_id,user_id)
visits = [
    (2,24),
    (2,24),
    (3,24),
    (3,24),
    (3,24),
    (4,12),
    (1,12),
    (1,12),
    (3,12),
    (5,12)
]

In [7]:
visitsRDD = sc.parallelize(visits)
# what do u meaaaaaaaaaaaAAAAAAAAAAAAAN

### Simil Parcial 2014-2 Recuperatorio
Se tiene un rdd con datos de la forma (url_id,user_id) representando visitas a un website. (Se pueden repetir registros). Se pide programar en PySpark un script que devuelva el usuario que mas sitios diferentes visita (no importa cuantas veces visita cada sitio).

In [6]:
visitsRDD.distinct()\
         .map(lambda x: (x[1], 1))\
         .reduceByKey(lambda a, b: a + b)\
         .reduce(lambda a, b: a if a[1] > b[1] else b)

(12, 4)